# 什么是容器

在 Python 中一切皆对象，对象的抽象就是类，而对象的集合就是容器。

列表（list: [0, 1, 2]），元组（tuple: (0, 1, 2)），字典（dict: {0:0, 1:1, 2:2}），集合（set: set([0, 1, 2])）都是容器。对于容器，你可以很直观地想象成多个元素在一起的单元；而不同容器的区别，正是在于内部数据结构的实现方法。然后，你就可以针对不同场景，选择不同时间和空间复杂度的容器。

# 什么是可迭代对象
所有的容器都是可迭代的（iterable）。这里的迭代，和枚举不完全一样。迭代可以想象成是你去买苹果，卖家并不告诉你他有多少库存。这样，每次你都需要告诉卖家，你要一个苹果，然后卖家采取行为：要么给你拿一个苹果；要么告诉你，苹果已经卖完了。你并不需要知道，卖家在仓库是怎么摆放苹果的。

可迭代对象，通过 iter() 函数返回一个迭代器，再通过 next() 函数就可以实现遍历。for in 语句将这个过程隐式化

# 什么是迭代器
严谨地说，迭代器（iterator）提供了一个 next 的方法。调用这个方法后，你要么得到这个容器的下一个对象，要么得到一个 StopIteration 的错误（苹果卖完了）。你不需要像列表一样指定元素的索引，因为字典和集合这样的容器并没有索引一说。比如，字典采用哈希表实现，那么你就只需要知道，next 函数可以不重复不遗漏地一个一个拿到所有元素即可。


我们来看下面这段代码，主要向你展示怎么判断一个对象是否可迭代。当然，这还有另一种做法，是 isinstance(obj, Iterable)

In [1]:
def is_iterable(param):
    try:
        iter(param)
        return True
    except TypeError:
        return False
    
params  = [ 1234, '1234', [1, 2, 3, 4], set([1, 2, 3, 4]), {1:1, 2:2, 3:3, 4:4}, (1, 2, 3, 4) ]

for param in params:
    print('{} is iterable? {}'.format(param, is_iterable(param)))

1234 is iterable? False
1234 is iterable? True
[1, 2, 3, 4] is iterable? True
{1, 2, 3, 4} is iterable? True
{1: 1, 2: 2, 3: 3, 4: 4} is iterable? True
(1, 2, 3, 4) is iterable? True


通过这段代码，你就可以知道，给出的类型中，除了数字 1234 之外，其它的数据类型都是可迭代的。

# 什么是生成器

生成器是懒人版本的迭代器。在迭代器中，如果我们想要枚举它的元素，这些元素需要事先生成。这里，我们先来看下面这个简单的样例。


In [2]:
import os
import psutil

def show_memory_info(hint):
    pid = os.getpid()
    p = psutil.Process(pid)
    
    info = p.memory_full_info()
    memory = info.uss / 1024. / 1024
    print('{} memory used: {} MB'.format(hint, memory))

In [3]:
def test_iterator():
    show_memory_info('initing iterator')
    list_1 = [i for i in range(100000000)]
    show_memory_info('after iterator initiated')
    print(sum(list_1))
    show_memory_info('after sum called')

def test_generator():
    show_memory_info('initing generator')
    list_2 = (i for i in range(100000000))  # 生成器
    show_memory_info('after generator initiated')
    print(sum(list_2))
    show_memory_info('after sum called')

%time test_iterator()
%time test_generator()



initing iterator memory used: 53.65234375 MB
after iterator initiated memory used: 3889.6171875 MB
4999999950000000
after sum called memory used: 3889.6171875 MB
CPU times: total: 6.64 s
Wall time: 7.87 s
initing generator memory used: 56.6484375 MB
after generator initiated memory used: 56.6484375 MB
4999999950000000
after sum called memory used: 56.6484375 MB
CPU times: total: 5.8 s
Wall time: 6.61 s


声明一个迭代器很简单，[i for i in range(100000000)]就可以生成一个包含一亿元素的列表。每个元素在生成后都会保存到内存中，你通过代码可以看到，它们占用了巨量的内存，内存不够的话就会出现 OOM 错误。 不过，我们并不需要在内存中同时保存这么多东西，比如对元素求和，我们只需要知道每个元素在相加的那一刻是多少就行了，用完就可以扔掉了。 于是，生成器的概念应运而生，在你调用 next() 函数的时候，才会生成下一个变量。**生成器在 Python 的写法是用小括号括起来**，(i for i in range(100000000))，即初始化了一个生成器。

这样一来，你可以清晰地看到，生成器并不会像迭代器一样占用大量内存，只有在被使用的时候才会调用。而且生成器在初始化的时候，并不需要运行一次生成操作，相比于 test_iterator() ，test_generator() 函数节省了一次生成一亿个元素的过程，因此耗时明显比迭代器短。

## 生成器示例

数学中有一个恒等式，(1 + 2 + 3 + ... + n)^2 = 1^3 + 2^3 + 3^3 + ... + n^3，想必你高中就应该学过它。现在，我们来验证一下这个公式的正确性。

In [4]:
def generator(k):
    i = 1
    while True:
        yield i ** k
        i += 1

gen_1 = generator(1)
gen_3 = generator(3)
print(gen_1)
print(gen_3)

def get_sum(n):
    sum_1, sum_3 = 0, 0
    for i in range(n):
        next_1 = next(gen_1)
        next_3 = next(gen_3)
        print('next_1 = {}, next_3 = {}'.format(next_1, next_3))
        sum_1 += next_1
        sum_3 += next_3
    print(sum_1 * sum_1, sum_3)

get_sum(8)


<generator object generator at 0x00000147D01C9080>
<generator object generator at 0x00000147D01C8FB0>
next_1 = 1, next_3 = 1
next_1 = 2, next_3 = 8
next_1 = 3, next_3 = 27
next_1 = 4, next_3 = 64
next_1 = 5, next_3 = 125
next_1 = 6, next_3 = 216
next_1 = 7, next_3 = 343
next_1 = 8, next_3 = 512
1296 1296


这段代码中，你首先注意一下 generator() 这个函数，它返回了一个生成器。

接下来的 yield 是魔术的关键。你可以理解为，函数运行到这一行的时候，程序会从这里暂停，然后跳出，不过跳到哪里呢？答案是 next() 函数。那么 i ** k 是干什么的呢？它其实成了 next() 函数的返回值。 这样，每次 next(gen) 函数被调用的时候，暂停的程序就又复活了，从 yield 这里向下继续执行；同时注意，局部变量 i 并没有被清除掉，而是会继续累加。我们可以看到 next_1 从 1 变到 8，next_3 从 1 变到 512。 

事实上，迭代器是一个有限集合，生成器则可以成为一个无限集。只管调用 next()，生成器根据运算会自动生成新的元素，然后返回给你，非常便捷。

In [7]:
# 常规写法，找到指定数值所在的index，并写入到result中
def index_normal(L, target):
    result = []
    for i, num in enumerate(L):
        if num == target:
            result.append(i)
    return result

print(index_normal([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2))


# 迭代器写法
def index_generator(L, target):
    for i, num in enumerate(L):
        if num == target:
            yield i

# index_generator 会返回一个 Generator 对象，需要使用 list 转换为列表后，才能用 print 输出。
print(list(index_generator([1, 6, 2, 4, 5, 2, 8, 6, 3, 2], 2)))

[2, 5, 9]
[2, 5, 9]


下面这个例子，给定两个序列，判定第一个是不是第二个的子序列。

序列就是列表，子序列则指的是，**一个列表的元素在第二个列表中都按顺序出现，但是并不必挨在一起**。举个例子，[1, 3, 5] 是 [1, 2, 3, 4, 5] 的子序列，[1, 4, 3] 则不是。

要解决这个问题，常规算法是贪心算法。我们维护两个指针指向两个列表的最开始，然后对第二个序列一路扫过去，如果某个数字和第一个指针指的一样，那么就把第一个指针前进一步。第一个指针移出第一个序列最后一个元素的时候，返回 True，否则返回 False。

In [12]:
def is_subsequence(a, b):
    # 将b转换成一个迭代器c
    c = iter(b)
    # 遍历a中的元素，判断每次遍历出来的元素是否在迭代器c中存在，返回True,False
    return all(i in c for i in a)


print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5])) 
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))

True
False


In [10]:
def is_subsequence(a, b):
    b = iter(b)
    print(b)

    gen = (i for i in a)
    print(gen)

    for i in gen:
        print(i)

    gen = ((i in b) for i in a)
    print(gen)

    for i in gen:
        print(i)

    return all(((i in b) for i in a))

print(is_subsequence([1, 3, 5], [1, 2, 3, 4, 5]))
print(is_subsequence([1, 4, 3], [1, 2, 3, 4, 5]))



<generator object is_subsequence.<locals>.<genexpr> at 0x00000147D01CACF0>
1
3
5
<generator object is_subsequence.<locals>.<genexpr> at 0x00000147D0202500>
True
True
True
False
<generator object is_subsequence.<locals>.<genexpr> at 0x00000147D01CACF0>
1
4
3
<generator object is_subsequence.<locals>.<genexpr> at 0x00000147D0202DC0>
True
True
False
False


首先，第二行的b = iter(b)，把列表 b 转化成了一个迭代器。 

接下来的gen = (i for i in a)产生一个生成器，这个生成器可以遍历对象 a，因此能够输出 1, 3, 5。

而 (i in b)大致等价于下面这段代码：
```python
while True:
    val = next(b)
    if val == i:
        yield True
```
最后的 all() 函数，用来判断一个迭代器的元素是否全部为 True，如果是则返回 True，否则就返回 False.

# 总结 
1. 容器是可迭代对象，可迭代对象调用 iter() 函数，可以得到一个迭代器。迭代器可以通过 next() 函数来得到下一个元素，从而支持遍历。 
2. 生成器是一种特殊的迭代器（注意这个逻辑关系反之不成立）。使用生成器，你可以写出来更加清晰的代码；合理使用生成器，可以降低内存占用、优化程序结构、提高程序速度。
3. 生成器在 Python 2 的版本上，是协程的一种重要实现方式；而 Python 3.5 引入 async await 语法糖后，生成器实现协程的方式就已经落后了。

# 思考题  
对于一个有限元素的生成器，如果迭代完成后，继续调用 next() ，会发生什么呢？生成器可以遍历多次吗？
- 生成器只能遍历一次，继续调用`next()` 会触发`raise StopIteration`。 
- 只有复位生成器才能重新进行遍历。
  